In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider
import xcoll.geometry.trajectories as tr
import xcoll.geometry.segments as seg
import xcoll.geometry.c_init as box


# %matplotlib ipympl
%matplotlib tk

# Bounding Box Tests

In [2]:
sintC = np.sin(np.deg2rad(45))
cosTC = np.cos(np.deg2rad(45))
rC = 2
sin_tb = np.sin(0.)
cos_tb = np.cos(0)
projL = rC*(cos_tb*cosTC + sin_tb*sintC)
projW = rC*(cos_tb*sintC - sin_tb*cosTC) 
b1 = box.BoundingBox(rC=rC, sin_tC=sintC, cos_tC=cosTC, sin_tb=sin_tb, cos_tb=cos_tb, l=2,w=2, proj_l=projL, proj_w=projW)


In [3]:
sintC = np.sin(np.deg2rad(11.309932))
cosTC = np.cos(np.deg2rad(11.309932))
rC = 5.099
sin_b = np.sin(np.deg2rad(45))
cos_b = np.cos(np.deg2rad(45))
projL = rC*(cos_b*cosTC + sin_b*sintC)
projW = rC*(cos_b*sintC - sin_b*cosTC) 
b2 = box.BoundingBox(rC=rC, sin_tC=sintC, cos_tC=cosTC, sin_tb=sin_b, cos_tb=cos_b, l=2.8284271247462,w=2.8284271247462, proj_l=projL, proj_w=projW)

In [4]:
sintC = np.sin(np.deg2rad(0.))
cosTC = np.cos(np.deg2rad(0.))
rC = 5.
sin_b = np.sin(np.deg2rad(45.))
cos_b = np.cos(np.deg2rad(45.))
projL = rC*(cos_b*cosTC + sin_b*sintC)
projW = rC*(cos_b*sintC - sin_b*cosTC) 
b3 = box.BoundingBox(rC=rC, sin_tC=sintC, cos_tC=cosTC, sin_tb=sin_b, cos_tb=cos_b, l=2.8284271247462,w=1.4142135623731, proj_l=projL, proj_w=projW)

In [10]:
tr.DriftTrajectory(s0=0, x0=0, theta=0).theta0

0.0

In [5]:
b1.theta

ValueError: Attribute theta not found in BoundingBox

In [7]:
box.BoundingBox.__dict__

mappingproxy({'__module__': 'xcoll.geometry.c_init.bounding_box',
              'rC': <field0 rC at 0>,
              'sin_tC': <field1 sin_tC at 8>,
              'cos_tC': <field2 cos_tC at 16>,
              'proj_l': <field3 proj_l at 24>,
              'proj_w': <field4 proj_w at 32>,
              'l': <field5 l at 40>,
              'w': <field6 w at 48>,
              'sin_tb': <field7 sin_tb at 56>,
              'cos_tb': <field8 cos_tb at 64>,
              '_kernels': {'overlaps': <xobjects.context.Kernel at 0x14af1de90>},
              '_needs_compilation': True,
              '_extra_c_sources': ['\n#ifndef XCOLL_GEOM_DEFINES_H\n#define XCOLL_GEOM_DEFINES_H\n#include <math.h>\n#include <stdio.h>\n#include <stdint.h>\n#include <stdlib.h>\n\n#ifndef XC_GEOM_EPSILON\n#define XC_GEOM_EPSILON 1e-15\n#endif\n\n#ifndef XC_GEOM_S_MAX\n#define XC_GEOM_S_MAX 1e+21\n#endif\n\n#ifndef XC_GEOM_ROOT_NEWTON_EPSILON\n#define XC_GEOM_ROOT_NEWTON_EPSILON 1e-10\n#endif\n\n#ifndef XC_GEOM_RO

In [6]:
overlap1 = b1.overlaps(b2=b2)  # this should overlap
overlap2 = b1.overlaps(b2=b3)  # this should not overlap
overlap3 = b2.overlaps(b2=b3)  # this should overlap

print(f"Overlap1: {overlap1}")
print(f"Overlap2: {overlap2}")
print(f"Overlap3: {overlap3}")

Compiling ContextCpu kernels...
Done compiling ContextCpu kernels.
Overlap1: 1
Overlap2: 0
Overlap3: 1


In [ ]:
fig, ax = plt.figure(figsize=(10,10))
ax.plot

In [ ]:
def test_BoundingBox_segment(segtype='line', **kwargs):
    if segtype == 'line':
        s1, x1, s2, x2 = kwargs['s1'], kwargs['x1'], kwargs['s2'], kwargs['x2']
        t1, t2 = kwargs.pop('t1', 0), kwargs.pop('t2', 1)
        L = seg.LineSegment(s1=s1,x1=x1,s2=s2,x2=x2)
        l = np.sqrt((s2 - s1)*(s2 - s1) + (x2 - x1)*(x2 - x1))
        w = L.box.l/3.
    elif segtype == 'halfopen':
        s1, x1, theta = kwargs['s1'], kwargs['x1'], kwargs['theta']
        L = seg.HalfOpenLineSegment(s1=s1,x1=x1,theta1=theta)
        t1, t2 = kwargs.pop('t1', 0), kwargs.pop('t2', 10)
        s2 = s1 + t2*np.cos(theta)
        x2 = x1 + t2*np.sin(theta)
        l = np.sqrt((s2 - s1)*(s2 - s1) + (x2 - x1)*(x2 - x1))
        w = L.box.l/3.
    elif segtype == 'circular':
        print("You can expect error at rC, projL and projW and cos/sin tC")
        R, sR, xR, theta1, theta2 = kwargs['R'], kwargs['sR'], kwargs['xR'], kwargs['theta1'], kwargs['theta2']
        L = seg.CircularSegment(R=R,sR=sR,xR=xR,theta1=theta1,theta2=theta2)
        t1, t2 = kwargs.pop('t1', 0), kwargs.pop('t2', 2*np.pi)
        x1 = xR + R*np.sin(theta1 + t1*(theta2 - theta1))
        s1 = sR + R*np.cos(theta1 + t1*(theta2 - theta1))
        x2 = xR + R*np.sin(theta1 + t2*(theta2 - theta1))
        s2 = sR + R*np.cos(theta1 + t2*(theta2 - theta1))
        print(f"Calculated: {s1}, {x1}, {s2}, {x2}")
        l = np.sqrt((s2 - s1)*(s2 - s1) + (x2 - x1)*(x2 - x1))
        w = R - np.sqrt(R*R - L.box.l*L.box.l/4.)
        rC = 1
        projL = 0. 
        projW = 0.

    elif segtype == 'bezier':
        s1, x1, cs1, cx1, cs2, cx2, s2, x2 = kwargs['s1'], kwargs['x1'], kwargs['cs1'], kwargs['cx1'], kwargs['cs2'], kwargs['cx2'], kwargs['s2'], kwargs['x2']
        L = seg.BezierSegment(s1=s1,x1=x1,cs1=cs1,cx1=cx1,cs2=cs2,cx2=cx2,s2=s2,x2=x2)
        t1, t2 = kwargs.pop('t1', 0), kwargs.pop('t2', 1)
        l = np.max(s1,s2) - np.min(s1,s2)
        w = np.max(x1,x2) - np.min(x1,x2)
        rC = np.sqrt(s1*s1 + s2*s2)
        projL = np.min(s1,s2)
        projW = np.min(x1,x2)
        sin_tc  = projW / rC
        cos_tc = projL / rC
        sin_tb = 0
        cos_tb = 1
    else:
        raise ValueError("Invalid segment type")

    if segtype != 'bezier':
        # Calculate sin_t and cos_t
        sin_t = (x2 - x1) / np.sqrt((x2 - x1)**2 + (s2 - s1)**2)
        cos_t = (s2 - s1) / np.sqrt((x2 - x1)**2 + (s2 - s1)**2)

        # Initialize sin_p and cos_p
        sin_p, cos_p = None, None

        # Adjust sin_t and cos_t if theta is larger than 180 degrees
        if sin_t < 0:  # if theta is larger than 180 degrees, theta = theta - 180
            sin_t = -sin_t
            cos_t = -cos_t

        # Determine sin_p and cos_p based on cos_t
        if cos_t < 1:  # if theta is larger than 90 degrees, phi = theta + 90
            sin_p = cos_t
            cos_p = -sin_t
        else:  # if theta is between 0 and 90 degrees, phi = theta - 90
            sin_p = -cos_t
            cos_p = sin_t

        rC =  np.sqrt( (s1+w/2.*cos_p) * (s1+w/2.*cos_p) + 
                    (x1+w/2.*sin_p) * (x1+w/2.*sin_p) )
        cos_tb = cos_t
        sin_tb = sin_t
        cos_tc = s1 / rC
        sin_tc = x1 / rC
        projL = rC*(cos_tb*cos_tc + sin_tb*sin_tc)
        projW = rC*(cos_tb*sin_tc - sin_tb*cos_tc)
    if abs(l - L.box.l) > 1e-9:
        print(f"Alert {segtype}: Length difference exceeds tolerance! Calculated: {l}, Box: {L.box.l}")
    if abs(w - L.box.w) > 1e-9:
        print(f"Alert {segtype}: Width difference exceeds tolerance! Calculated: {w}, Box: {L.box.w}")
    if abs(sin_tc - L.box.sin_tC) > 1e-9:
        print(f"Alert {segtype}: sin_tC difference exceeds tolerance! Calculated: {sin_tc}, Box: {L.box.sin_tC}")
    if abs(cos_tc - L.box.cos_tC) > 1e-9:
        print(f"Alert {segtype}: cos_tC difference exceeds tolerance! Calculated: {cos_tc}, Box: {L.box.cos_tC}")
    if abs(sin_tb - L.box.sin_tb) > 1e-9:
        print(f"Alert {segtype}: sin_tb difference exceeds tolerance! Calculated: {sin_tb}, Box: {L.box.sin_tb}")
    if abs(cos_tb - L.box.cos_tb) > 1e-9:
        print(f"Alert {segtype}: cos_tb difference exceeds tolerance! Calculated: {cos_tb}, Box: {L.box.cos_tb}")
    if abs(projL - L.box.proj_l) > 1e-9:
        print(f"Alert {segtype}: proj_l difference exceeds tolerance! Calculated: {projL}, Box: {L.box.proj_l}")
    if abs(projW - L.box.proj_w) > 1e-9:
        print(f"Alert {segtype}: proj_w difference exceeds tolerance! Calculated: {projW}, Box: {L.box.proj_w}")
    if abs(rC - L.box.rC) > 1e-9:
        print(f"Alert {segtype}: rC difference exceeds tolerance! Calculated: {rC}, Box: {L.box.rC}")
    print()


In [4]:
test_BoundingBox_segment(segtype='line', s1=0, x1=0, s2=1, x2=1, t1=0, t2=1)
test_BoundingBox_segment(segtype='halfopen', s1=1, x1=0, theta=np.pi/3, t1=0, t2=10)
test_BoundingBox_segment(segtype='circular', R=1, sR=0, xR=0, theta1=0, theta2=np.pi/2)



You can expect error at rC, projL and projW and cos/sin tC
Alert circular: Length difference exceeds tolerance! Calculated: 1.950735944167263, Box: 0.0

tt1 = 0.000000, tt2 = 0.000000


In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider
import xcoll.geometry.trajectories as tr
import xcoll.geometry.segments as seg
import xcoll.geometry.c_init as box


# %matplotlib ipympl
%matplotlib tk

In [3]:
L = seg.HalfOpenLineSegment(s1=1,x1=0,cos_t1=np.sqrt(2)/2)
#L.box.l

#L1 = seg.LineSegment(s1=0,x1=0,s2=1,x2=1)
#L1.box.l

{'s1': 1, 'x1': 0, 'cos_t1': np.float64(0.7071067811865476)}


# Trajectory Tests

In [ ]:
kwargs = {'s0': [-3, 3, 0], 'x0': [-3, 3, 0], 'xp': [np.deg2rad(-45), np.deg2rad(45), np.deg2rad(0.5)]}

tr.DriftTrajectory._inspect(**kwargs)

In [ ]:
kwargs = {'sR': [-3, 3, 0.2], 'xR': [-3, 3, -0.3], 's0': [-2, 2, 0], 'x0': [-2, 2, 0]}
tr.CircularTrajectory._inspect(**kwargs)

In [ ]:
ran_1 = np.random.normal()
ran_2 = np.random.normal()

fig, _ = tr.MultipleCoulombTrajectory(s0=0.1, x0=1, xp=np.tan(np.deg2rad(30.)), pc=1e9, beta=1.e9/np.sqrt(0.938e9**2 + 1.e9**2),
                                      q=1, X0=0.0001, ran_1=ran_1, ran_2=ran_2).plot()

In [ ]:
fig, _ =  tr.CircularTrajectory(s0=0, x0=-1, sR=0, xR=0).plot(l1=-0.5, l2=np.pi/2-0.01)


# Segment Tests

In [ ]:
kwargs = {'s1': [-3, 3, 0], 'x1': [-3, 3, 0], 's2': [-3, 3, 1], 'x2': [-3, 3, 1]}

seg.LineSegment._inspect(**kwargs)

In [ ]:
kwargs = {'s1': [-3, 3, 0], 'x1': [-3, 3, 0], 'theta1': [-2*np.pi, 2*np.pi, np.pi/3]}

seg.HalfOpenLineSegment._inspect(**kwargs)

In [ ]:
kwargs = {'R': [0.1, 3, 1.2], 'sR': [-3, 3, 0.2], 'xR': [-3, 3, -0.3],
          'theta1': [-2*np.pi, 2*np.pi, -np.pi/3], 'theta2': [-2*np.pi, 2*np.pi, np.pi/3],}

seg.CircularSegment._inspect(**kwargs)
# 1. Providing the centre, radius, and angles: CircularSegment(R=..., sR=..., xR=..., theta1=..., theta2=...)
# 2. Providing the start point, radius, and angles: CircularSegment(R=..., s1=..., x1=..., theta1=..., theta2=...)
# 3. Providing the end point, radius, and angles: CircularSegment(R=..., s2=..., x2=..., theta1=..., theta2=...)
# 4. Providing the start and end point, and the (possibly negative) curvature: CircularSegment(k=..., s1=..., x1=..., s2=..., x2=...)
# 5. Providing the centre, start point, and an angular shift: CircularSegment(sR=..., xR=..., s1=..., x1=..., delta_theta=...)
# 6. Providing the centre, end point, and an angular shift: CircularSegment(sR=..., xR=..., s2=..., x2=..., delta_theta=...)

In [ ]:
kwargs = {'R': [0.1, 3, 1.2], 's1': [-3, 3, 0.2], 'x1': [-3, 3, -0.3],
          'theta1': [-2*np.pi, 2*np.pi, -np.pi/3], 'theta2': [-2*np.pi, 2*np.pi, np.pi/3],}

seg.CircularSegment._inspect(**kwargs)

In [ ]:
kwargs = {'R': [0.1, 3, 1.2], 's2': [-3, 3, 0.2], 'x2': [-3, 3, -0.3],
          'theta1': [-2*np.pi, 2*np.pi, -np.pi/3], 'theta2': [-2*np.pi, 2*np.pi, np.pi/3],}

seg.CircularSegment._inspect(**kwargs)

In [ ]:
kwargs = {'k': [-1, 1, 0.1], 's1': [-3, 3, 0.2], 'x1': [-3, 3, -0.3], 's2': [-3, 3, 0.7], 'x2': [-3, 3, 0.5]}

# seg.CircularSegment._inspect(**kwargs)
seg.CircularSegment._inspect(plot_bounding_box=False, plot_control_points=False, **kwargs)

In [ ]:
kwargs = {'sR': [-3, 3, 0.2], 'xR': [-3, 3, -0.3], 's1': [-3, 3, 0.7], 'x1': [-3, 3, 0.5],
          'delta_theta': [-2*np.pi, 2*np.pi, np.pi/3]}

seg.CircularSegment._inspect(**kwargs)

In [ ]:
kwargs = {'sR': [-3, 3, 0.2], 'xR': [-3, 3, -0.3], 's1': [-3, 3, 0.7], 'x1': [-3, 3, 0.5],
          'delta_theta': [-2*np.pi, 2*np.pi, -np.pi/3]}

In [ ]:
kwargs = {
    's1': [-3, 3, 0],
    'x1': [-3, 3, 0],
    'cs1': [-3, 3, 0],
    'cx1': [-3, 3, 1],
    'cs2': [-3, 3, 1],
    'cx2': [-3, 3, 0],
    's2': [-3, 3, 1],
    'x2': [-3, 3, 1],
}

seg.BezierSegment._inspect(**kwargs)

# Shape

In [ ]:
%load_ext autoreload
%autoreload 2
# %load_ext wurlitzer
%matplotlib ipympl

import numpy as np
import xobjects as xo

import xcoll as xc
from xcoll.geometry.segments import get_max_crossings
from xcoll.geometry.trajectories import DriftTrajectory

import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [ ]:
def check_drift(shape, plot_object=False, xlim=(-0.5, 1.5), ylim=(-0.5, 1.5)):
    to_remove = []
    s_val = np.linspace(*xlim, 200)
    fig, ax = plt.subplots(1, 1, figsize=(8,8))
    if plot_object:
        shape.plot(axes=ax)
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)
    ax.set_aspect('equal')

    n_hit = np.zeros(1, dtype=np.int8)
    s = np.zeros(get_max_crossings(shape, DriftTrajectory), dtype=np.float64)
    # s = np.zeros(8, dtype=np.float64)

    def update_plot(s0, x0, part_theta):
        # Clean the previous trajectory and points
        while to_remove:
            last_point = to_remove.pop()  # Get the last added point
            last_point.remove()
        m = np.tan(np.radians(part_theta))
        # Draw drift trajectory and point
        spread = 10 # degrees
        line, = ax.plot(s_val, m*(s_val - s0) + x0, c='k')
        to_remove.append(line)
        scatter = ax.scatter(s0, x0, c='r', s=20)
        to_remove.append(scatter)
        m1 = np.tan(np.radians(part_theta+spread))
        m2 = np.tan(np.radians(part_theta-spread))
        line = ax.fill_between(s_val, m1*(s_val - s0) + x0, m2*(s_val - s0) + x0, alpha=0.2)
        to_remove.append(line)
        # Create hits for different slopes (small cone around the trajectory)
        hits_s = []
        hits_x = []
        for sp in np.linspace(-spread, spread, 100):
            n_hit[0] = 0
            m = np.tan(np.radians(part_theta + sp))
            if isinstance(shape, xc.Shape2DV):
                shape.crossing_drift(n_hit=n_hit, s=s, s0=s0, x0=x0, xm=m, y0=0, ym=0)
            else:
                shape.crossing_drift(n_hit=n_hit, s=s, s0=s0, x0=x0, xm=m)
            hits_s += list(s[:n_hit[0]])
            hits_x += list(x0 + m*(s[:n_hit[0]] - s0))
        # for i in range(n_hit[0]):
        scatter = ax.scatter(hits_s, hits_x, c='g', s=5)
        # to_remove.append(scatter)
        fig.canvas.draw()  # Redraw the canvas to show the new point


    def onclick(event):
        s0, x0 = event.xdata, event.ydata
        if s0 is not None and x0 is not None:  # Ensure the click is inside the plot
            update_plot(s0, x0, part_theta_slider.value)


    fig.canvas.mpl_connect('button_press_event', onclick)
    part_theta_slider = widgets.IntSlider(min=-89, max=89, step=1, value=0)

    @interact(part_theta=part_theta_slider)
    def check_linesegments(part_theta):
        if to_remove:
            scatter = to_remove[1]  # The second point is the clicked scatter point
            s0, x0 = scatter.get_offsets()[0, 0], scatter.get_offsets()[0, 1]

            # Update the plot with the new slope based on the slider
            update_plot(s0, x0, part_theta)

In [ ]:
shape1 = xc.Shape2D([xc.LineSegment(s1=0, x1=0, s2=0.2, x2=1), xc.LineSegment(s1=0.2, x1=1, s2=1, x2=1),
                     xc.LineSegment(s1=1, x1=1, s2=0.8, x2=0), xc.LineSegment(s1=0.8, x1=0, s2=0, x2=0)])
shape2 = xc.Shape2D([xc.LineSegment(s1=0, x1=0, s2=0.2, x2=1),
                     xc.BezierSegment(s1=0.2, x1=1, s2=1+0.5*np.cos(3*np.pi/4), x2=0.5+0.5*np.sin(3*np.pi/4), cs1=0.5, cx1=2.5, cs2=1+0.5*np.cos(3*np.pi/4)-0.6, cx2=0.5+0.5*np.sin(3*np.pi/4)-0.6),
                     xc.CircularSegment(R=0.5, s=1, x=0.5, t1=-np.pi/2, t2=3*np.pi/4),
                     xc.LineSegment(s1=1, x1=0, s2=0, x2=0),
                     xc.HalfOpenLineSegment(s=2, x=1, t=np.pi/4),
                     xc.LineSegment(s1=2, x1=1, s2=3, x2=0),
                     xc.HalfOpenLineSegment(s=3, x=0, t=np.pi/4)])
shape3 = xc.Shape2DV([xc.LineSegment(s1=0, x1=0, s2=0.2, x2=1),
                      xc.BezierSegment(s1=0.2, x1=1, s2=1+0.5*np.cos(3*np.pi/4), x2=0.5+0.5*np.sin(3*np.pi/4), cs1=0.5, cx1=2.5, cs2=1+0.5*np.cos(3*np.pi/4)-0.6, cx2=0.5+0.5*np.sin(3*np.pi/4)-0.6),
                      xc.CircularSegment(R=0.5, s=1, x=0.5, t1=-np.pi/2, t2=3*np.pi/4),
                      xc.LineSegment(s1=1, x1=0, s2=0, x2=0),
                      xc.HalfOpenLineSegment(s=2, x=1, t=np.pi/4),
                      xc.LineSegment(s1=2, x1=1, s2=3, x2=0),
                      xc.HalfOpenLineSegment(s=3, x=0, t=np.pi/4),
                      xc.HalfOpenLineSegment(s=1.8, x=-0.5, t=5*np.pi/8),
                      xc.CircularSegment(R=0.3, s=1.8-0.3*np.cos(np.pi/8), x=-0.5-0.3*np.sin(np.pi/8), t1=-7*np.pi/8, t2=np.pi/8),
                      xc.HalfOpenLineSegment(s=1.8-0.6*np.cos(np.pi/8), x=-0.5-0.6*np.sin(np.pi/8), t=5*np.pi/8)],
                     vlimit=[-0.1, 0.1])
shape4 = xc.Shape2DV([xc.CircularSegment(R=1, s=0, x=0, t1=0, t2=2*np.pi/3),
                      xc.CircularSegment(R=1, s=0, x=0, t1=2*np.pi/3, t2=4*np.pi/3),
                      xc.CircularSegment(R=1, s=0, x=0, t1=4*np.pi/3, t2=2*np.pi)],
                     vlimit=[-0.1, 0.1])
R1 = 0.5
R2 = 2
t = np.arcsin(R1/R2)
shape5 = xc.Shape2D([xc.CircularSegment(R=R1, s=0, x=0, t1=np.pi/2, t2=-np.pi/2),
                     xc.CircularSegment(R=R2, s=-R2*np.cos(t), x=0, t1=-t, t2=t)])
shape6 = xc.Shape2D([xc.HalfOpenLineSegment(s=0, x=0, t=np.pi/4),
                     xc.HalfOpenLineSegment(s=0, x=0, t=-np.pi/4)])

In [ ]:
#This is meant to fail

xc.Shape2D([xc.LineSegment(s1=0, x1=0, s2=0.2, x2=1), xc.LineSegment(s1=0.2, x1=1, s2=1, x2=1), xc.LineSegment(s1=0.2, x1=1, s2=1, x2=1),
            xc.LineSegment(s1=1, x1=1, s2=0.8, x2=0), xc.LineSegment(s1=0.8, x1=0, s2=0, x2=0)]).get_vertices()

In [ ]:
fig, _ = shape1.plot3d(); fig.show()
fig, _ = shape2.plot3d(); fig.show()
fig, _ = shape3.plot3d(); fig.show()
fig, _ = shape4.plot3d(); fig.show()
fig, _ = shape5.plot3d(); fig.show()
fig, _ = shape6.plot3d(); fig.show()

In [ ]:
shape_1 = xc.Shape2D([xc.LineSegment(s1=0, x1=0, s2=0.2, x2=1), xc.LineSegment(s1=0.2, x1=1, s2=1, x2=1),
                     xc.LineSegment(s1=1, x1=1, s2=0.8, x2=0), xc.LineSegment(s1=0.8, x1=0, s2=0, x2=0)])

check_drift(shape_1, plot_object=False)

In [ ]:
shape_2 = xc.Shape2D([xc.LineSegment(s1=0, x1=0, s2=0.2, x2=1),
                     xc.BezierSegment(s1=0.2, x1=1, s2=1+0.5*np.cos(3*np.pi/4), x2=0.5+0.5*np.sin(3*np.pi/4), cs1=0.5, cx1=2.5, cs2=1+0.5*np.cos(3*np.pi/4)-0.6, cx2=0.5+0.5*np.sin(3*np.pi/4)-0.6),
                     xc.CircularSegment(R=0.5, s=1, x=0.5, t1=-np.pi/2, t2=3*np.pi/4),
                     xc.LineSegment(s1=1, x1=0, s2=0, x2=0),
                     xc.HalfOpenLineSegment(s=2, x=1, t=np.pi/4),
                     xc.LineSegment(s1=2, x1=1, s2=3, x2=0),
                     xc.HalfOpenLineSegment(s=3, x=0, t=np.pi/4)])
check_drift(shape_2, xlim=(-0.5, 4), ylim=(-0.5,2), plot_object=True)

In [ ]:
# seg = xc.CircularSegment(R=1, s=0., x=0., t1=0,        t2=np.pi/2)
# seg = xc.CircularSegment(R=1, s=0., x=0., t1=-np.pi/2, t2=np.pi/2)
# seg = xc.CircularSegment(R=1, s=0., x=0., t1=np.pi/2,  t2=-np.pi/2)
# seg = xc.CircularSegment(R=1, s=0., x=0., t1=0,        t2=3*np.pi/2)
# seg = xc.CircularSegment(R=1, s=0., x=0., t1=0,        t2=2*np.pi)
# seg = xc.CircularSegment(R=1, s=0., x=0., t1=0,        t2=-np.pi/2)
# seg = xc.CircularSegment(R=1, s=0., x=0., t1=-np.pi/2, t2=0)
seg = xc.CircularSegment(R=1, s=0., x=0., t1=3*np.pi/2,t2=0)

check_drift(xc.Shape2D([seg]), plot_object=True, xlim=(-1.5, 1.5), ylim=(-1.5, 1.5))

In [ ]:
shape_3 = xc.Shape2DV([xc.LineSegment(s1=0, x1=0, s2=0.2, x2=1),
                      xc.BezierSegment(s1=0.2, x1=1, s2=1+0.5*np.cos(3*np.pi/4), x2=0.5+0.5*np.sin(3*np.pi/4), cs1=0.5, cx1=2.5, cs2=1+0.5*np.cos(3*np.pi/4)-0.6, cx2=0.5+0.5*np.sin(3*np.pi/4)-0.6),
                      xc.CircularSegment(R=0.5, s=1, x=0.5, t1=-np.pi/2, t2=3*np.pi/4),
                      xc.LineSegment(s1=1, x1=0, s2=0, x2=0),
                      xc.HalfOpenLineSegment(s=2, x=1, t=np.pi/4),
                      xc.LineSegment(s1=2, x1=1, s2=3, x2=0),
                      xc.HalfOpenLineSegment(s=3, x=0, t=np.pi/4)],
                    vlimit=[-0.1, 0.1])
check_drift(shape_3, xlim=(-0.5, 4), ylim=(-0.5,2), plot_object=True)